# FINAL Dataframe-  NLS -  Encyclopaedia Britannica

This notebook is going to merge all the data available that we have from our postprocess dataframes.

For each postprocess edition dataframe that we got from **Merging_EB_Terms.ipynb** (e.g. results_eb_1_edition_dataframe, results_eb_2_edition_dataframe, etc) we are going to add the information from the dataframe that we got from **Metadata_EB.ipynb** (metadata_eb_dataframe). 

The idea is to have per edition dataframe (and also supplement dataframe), all the information (which currently is splitted across several dataframes) in one. 


This notebook will store the final dataframes in results_NLS directory, and their name schema will be **final_eb_< NUM_EDITION >_dataframe**.

Per entry in these new dataframes we will have the following columns (see an example of one entry of the first edition):

- MMSID:                                              
- editionTitle:                          First edition, 1771, Volume 1, A-B
- editor:                                                  Smellie, William
- editor_date:                                                   1740-1795
- genre:                                                       encyclopedia
- language:                                                             eng
- termsOfAddress:                                                       NaN
- numberOfPages:                                                        832
- physicalDescription:               3 v., 160 plates : ill. ; 26 cm. (4to)
- place:                                                         Edinburgh
- publisher:              Printed for A. Bell and C. Macfarquhar; and so...
- referencedBy:           [Alston, R.C.  Engl. language III, 560, ESTC T...
- shelfLocator:                                                        EB.1
- editionSubTitle:        Illustrated with one hundred and sixty copperp...
- volumeTitle:            Encyclopaedia Britannica; or, A dictionary of ...
- year:                                                                1771
- volumeId:                                                       144133901
- metsXML:                                               144133901-mets.xml
- permanentURL:                            https://digital.nls.uk/144133901
- publisherPersons:                     [C. Macfarquhar, Colin Macfarquhar]
- volumeNum:                                                              1
- letters:                                                              A-B
- part:                                                                   0
- editionNum:                                                             1
- supplementTitle:                                                         
- supplementSubTitle:                                                      
- supplementsTo:                                                         []
- numberOfVolumes:                                                        6
- term:                                                                  OR
- definition:             A NEW A D I C T I A A, the name of several riv...
- relatedTerms:                                                          []
- header:                                           EncyclopaediaBritannica
- startsAt:                                                              15
- endsAt:                                                                15
- numberOfTerms:                                                         22
- numberOfWords:                                                         54
- positionPage:                                                           0
- typeTerm:                                                         Article
- altoXML:                                  144133901/alto/188082904.34.xml

### Loading the necessary libraries

In [1]:
import yaml
import numpy as np
import collections
import string
import copy


In [2]:
import pandas as pd
from yaml import safe_load
from pandas.io.json import json_normalize
from difflib import SequenceMatcher

### Functions

In [3]:
def add_metadata(df):
    
    mask = df["editionTitle"].str.contains('Volume')
    for i in range(0, len (mask)):
        if mask[i]:
            tmp=df.loc[i,'editionTitle'].split("Volume ")[1].split(",")
            if len(tmp)>=1:
                volume= tmp[0]
                letters = tmp[-1].replace(" ","")
                part_tmp = volume.split("Part ")
                if len(part_tmp)>1:
                    volume=part_tmp[0]
                    part = part_tmp[1]
    
                    try:
                        part = int(part)
                    except:
                        if "I" in part:
                            part = 1
                else:
                    part=0

                volume = int(volume)
                df.loc[i, "letters"] = letters
                df.loc[i,"part"] = part
                df.loc[i ,"volumeNum"] = volume
    return df

In [4]:
def create_dataframe(df_metadata, df_data, editionNum):
    
    
    appended_data = []
    df_m_ed = df_metadata[df_metadata['editionNum'] == editionNum]
    list_years=df_m_ed["year"].unique()
    
    for y in range(0, len(list_years)):
        df_d_year=df_data[df_data['year'] == list_years[y]].reset_index(drop=True)
        df_m_year = df_m_ed[df_m_ed['year'] == list_years[y]].reset_index(drop=True)
        list_vols = df_m_year["volumeNum"].unique()
        print("YEAR %s, list_vols %s" % (list_years[y], list_vols))
        for v in range(0,len(list_vols)):
            df_d_year_vl=df_d_year[df_d_year["volumeNum"] == list_vols[v]].reset_index(drop=True)
            df_m_year_vl = df_m_year[df_m_year['volumeNum']==list_vols[v]].reset_index(drop=True)
            list_parts = df_m_year_vl["part"].unique()
            print("VOL %s, list_parts are %s" % (list_vols[v], list_parts))
            for p in range(0, len(list_parts)):
                df_d_year_vl_p=df_d_year_vl[df_d_year_vl["part"] == list_parts[p]].reset_index(drop=True)
                df_m_year_vl_p=df_m_year_vl[df_m_year_vl["part"] == list_parts[p]].reset_index(drop=True)
            
                
                d_rows_year_vl_p=len(df_d_year_vl_p.index)-1
                print("number of data rows for year %s, vol %s and part %s is %s" % (list_years[y], list_vols[v], list_parts[p], d_rows_year_vl_p ))
        
                df_m_year_vl_p_f = df_m_year_vl_p.append([df_m_year_vl_p]*d_rows_year_vl_p,ignore_index=True).reset_index(drop=True)
                result = pd.concat([df_m_year_vl_p_f, df_d_year_vl_p], axis=1).reset_index(drop=True)
                print("Len of the new temporal concatenated df %s" %len(result.index))
                appended_data.append(result)
    
    if editionNum == 3:
        df_m_sup = df_m[df_m['supplementTitle'].str.contains("third")]
        list_years=df_m_sup["year"].unique()
        
        for y in range(0, len(list_years)):
            df_d_year=df_data[df_data['year'] == list_years[y]].reset_index(drop=True)
            df_m_year = df_m_sup[df_m_sup['year'] == list_years[y]].reset_index(drop=True)
            list_vols = df_m_year["volumeNum"].unique()
            print("SUP YEAR %s, list_vols %s" % (list_years[y], list_vols))
            for v in range(0,len(list_vols)):
                df_d_year_vl=df_d_year[df_d_year["volumeNum"] == list_vols[v]].reset_index(drop=True)
                df_m_year_vl = df_m_year[df_m_year['volumeNum']==list_vols[v]].reset_index(drop=True)
                list_parts = df_m_year_vl["part"].unique()
                print("SUP VOL %s, list_parts are %s" % (list_vols[v], list_parts))
                for p in range(0, len(list_parts)):
                    df_d_year_vl_p=df_d_year_vl[df_d_year_vl["part"] == list_parts[p]].reset_index(drop=True)
                    df_m_year_vl_p=df_m_year_vl[df_m_year_vl["part"] == list_parts[p]].reset_index(drop=True)
            
                
                    d_rows_year_vl_p=len(df_d_year_vl_p.index)-1
                    print("SUP number of data rows for year %s, vol %s and part %s is %s" % (list_years[y], list_vols[v], list_parts[p], d_rows_year_vl_p ))
        
                    df_m_year_vl_p_f = df_m_year_vl_p.append([df_m_year_vl_p]*d_rows_year_vl_p,ignore_index=True).reset_index(drop=True)
                    result = pd.concat([df_m_year_vl_p_f, df_d_year_vl_p], axis=1).reset_index(drop=True)
                    print("SUP Len of the new temporal concatenated df %s" %len(result.index))
                    appended_data.append(result)
                
                
    final_df = pd.concat(appended_data).reset_index(drop=True)
    final_df =  final_df.loc[:,~final_df.columns.duplicated()]
    return final_df

In [5]:
def create_dataframe_sup(df_metadata, df_data):

    appended_data=[]
    df_m_sup = df_m[df_m['supplementTitle'].str.contains("fourth")]
    list_years=df_m_sup["year"].unique()
        
    for y in range(0, len(list_years)):
        df_d_year=df_data[df_data['year'] == list_years[y]].reset_index(drop=True)
        df_m_year = df_m_sup[df_m_sup['year'] == list_years[y]].reset_index(drop=True)
        list_vols = df_m_year["volumeNum"].unique()
        print("SUP YEAR %s, list_vols %s" % (list_years[y], list_vols))
        for v in range(0,len(list_vols)):
            df_d_year_vl=df_d_year[df_d_year["volumeNum"] == list_vols[v]].reset_index(drop=True)
            df_m_year_vl = df_m_year[df_m_year['volumeNum']==list_vols[v]].reset_index(drop=True)
            list_parts = df_m_year_vl["part"].unique()
            print("SUP VOL %s, list_parts are %s" % (list_vols[v], list_parts))
            for p in range(0, len(list_parts)):
                df_d_year_vl_p=df_d_year_vl[df_d_year_vl["part"] == list_parts[p]].reset_index(drop=True)
                df_m_year_vl_p=df_m_year_vl[df_m_year_vl["part"] == list_parts[p]].reset_index(drop=True)
            
                
                d_rows_year_vl_p=len(df_d_year_vl_p.index)-1
                print("SUP number of data rows for year %s, vol %s and part %s is %s" % (list_years[y], list_vols[v], list_parts[p], d_rows_year_vl_p ))
        
                df_m_year_vl_p_f = df_m_year_vl_p.append([df_m_year_vl_p]*d_rows_year_vl_p,ignore_index=True).reset_index(drop=True)
                result = pd.concat([df_m_year_vl_p_f, df_d_year_vl_p], axis=1).reset_index(drop=True)
                print("SUP Len of the new temporal concatenated df %s" %len(result.index))
                appended_data.append(result)
                
                
    final_df = pd.concat(appended_data).reset_index(drop=True)
    final_df =  final_df.loc[:,~final_df.columns.duplicated()]
    return final_df

### 1. Loading the metadata dataframe

In [6]:
df_m= pd.read_json(r'../../results_NLS/metadata_eb_dataframe', orient="index")

In [7]:
df_m[df_m["editionNum"]==2]

,MMSID,editionTitle,editor,editor_date,genre,language,termsOfAddress,numberOfPages,physicalDescription,place,...,permanentURL,publisherPersons,volumeNum,letters,part,editionNum,supplementTitle,supplementSubTitle,supplementsTo,numberOfVolumes
6,997902523804341,"Second edition, Volume 1, A-AST",None,None,encyclopedia,eng,None,872,"10v.,plates : maps ; 4to",Edinburgh,...,https://digital.nls.uk/144850370,"[W. Gordon, J. Bell, J. Dickson, C. Elliot]",1,A-AST,0,2,,,[],10
7,997902523804341,"Second edition, Volume 2, Astronomy-BZO",None,None,encyclopedia,eng,None,882,"10v.,plates : maps ; 4to",Edinburgh,...,https://digital.nls.uk/144850373,"[W. Gordon, J. Bell, J. Dickson, C. Elliot]",2,Astronomy-BZO,0,2,,,[],10
8,997902523804341,"Second edition, Volume 3, C",None,None,encyclopedia,eng,None,872,"10v.,plates : maps ; 4to",Edinburgh,...,https://digital.nls.uk/144850374,"[W. Gordon, J. Bell, J. Dickson, C. Elliot]",3,C,0,2,,,[],10
9,997902523804341,"Second edition, Volume 4, D-F",None,None,encyclopedia,eng,None,868,"10v.,plates : maps ; 4to",Edinburgh,...,https://digital.nls.uk/144850375,"[W. Gordon, J. Bell, J. Dickson, C. Elliot]",4,D-F,0,2,,,[],10
10,997902523804341,"Second edition, Volume 5, G-J",None,None,encyclopedia,eng,None,980,"10v.,plates : maps ; 4to",Edinburgh,...,https://digital.nls.uk/144850376,"[W. Gordon, J. Bell, J. Dickson, C. Elliot]",5,G-J,0,2,,,[],10
11,997902523804341,"Second edition, Volume 6, K-Medicine",None,None,encyclopedia,eng,None,1062,"10v.,plates : maps ; 4to",Edinburgh,...,https://digital.nls.uk/144850377,"[W. Gordon, J. Bell, J. Dickson, C. Elliot]",6,K-Medicine,0,2,,,[],10
12,997902523804341,"Second edition, Volume 7, Medicines-Optics",None,None,encyclopedia,eng,None,886,"10v.,plates : maps ; 4to",Edinburgh,...,https://digital.nls.uk/144850378,"[W. Gordon, J. Bell, J. Dickson, C. Elliot]",7,Medicines-Optics,0,2,,,[],10
13,997902523804341,"Second edition, Volume 8, Optics-Poetry",None,None,encyclopedia,eng,None,872,"10v.,plates : maps ; 4to",Edinburgh,...,https://digital.nls.uk/144850379,"[W. Gordon, J. Bell, J. Dickson, C. Elliot]",8,Optics-Poetry,0,2,,,[],10
24,997902523804341,"Second edition, Volume 9, POI-SCU",None,None,encyclopedia,eng,None,874,"10v.,plates : maps ; 4to",Edinburgh,...,https://digital.nls.uk/190273289,"[W. Gordon, J. Bell, J. Dickson, C. Elliot]",9,POI-SCU,0,2,,,[],10
25,997902523804341,"Second edition, Volume 10, SCU-Appendix",None,None,encyclopedia,eng,None,1280,"10v.,plates : maps ; 4to",Edinburgh,...,https://digital.nls.uk/190273290,"[W. Gordon, J. Bell, J. Dickson, C. Elliot]",10,SCU-Appendix,0,2,,,[],10


### 2. Creating the NEW final dataframes for all Editions


Possible values:
- results_eb_4_5_6_suplement_dataframe
- results_eb_8_edition_dataframe
- results_eb_7_edition_dataframe
- results_eb_6_edition_dataframe
- results_eb_5_edition_dataframe
- results_eb_4_edition_dataframe
- results_eb_3_edition_dataframe
- results_eb_2_edition_dataframe
- results_eb_1_edition_dataframe


In [8]:
df_d = pd.read_json("../../results_NLS/results_eb_2_edition_dataframe", orient="index") 
len(df_d)

16665

In [9]:
df_d.columns

Index(['term', 'definition', 'relatedTerms', 'header', 'startsAt', 'endsAt',
       'numberOfTerms', 'numberOfWords', 'numberOfPages', 'positionPage',
       'typeTerm', 'editionTitle', 'editionNum', 'supplementTitle',
       'supplementsTo', 'year', 'place', 'volumeTitle', 'volumeNum', 'letters',
       'part', 'altoXML'],
      dtype='object')

In [10]:
df_d.tail(3)["volumeNum"]

16662    9
16663    9
16664    9
Name: volumeNum, dtype: int64

Note: add_metadata function can take a while. 

In [11]:
df_d = add_metadata(df_d)

**IMPORTANT**

- Use create_dataframe for creating the final dataframe for all editions. This function needs to **indicate the NUMBER of the edition, in the last argument**. 
- Use create_dataframe_sup for creating the final dataframe for the supplements. 

In [12]:
df_final=create_dataframe(df_m, df_d, 2)
#df_final=create_dataframe_sup(df_m, df_d)

YEAR 1778, list_vols [ 1  2  3  4  5  6  7  8  9 10]
VOL 1, list_parts are [0]
number of data rows for year 1778, vol 1 and part 0 is 2242
Len of the new temporal concatenated df 2243
VOL 2, list_parts are [0]
number of data rows for year 1778, vol 2 and part 0 is 1760
Len of the new temporal concatenated df 1761
VOL 3, list_parts are [0]
number of data rows for year 1778, vol 3 and part 0 is 2250
Len of the new temporal concatenated df 2251
VOL 4, list_parts are [0]
number of data rows for year 1778, vol 4 and part 0 is 1894
Len of the new temporal concatenated df 1895
VOL 5, list_parts are [0]
number of data rows for year 1778, vol 5 and part 0 is 1841
Len of the new temporal concatenated df 1842
VOL 6, list_parts are [0]
number of data rows for year 1778, vol 6 and part 0 is 1154
Len of the new temporal concatenated df 1155
VOL 7, list_parts are [0]
number of data rows for year 1778, vol 7 and part 0 is 1005
Len of the new temporal concatenated df 1006
VOL 8, list_parts are [0]
numb

And these are the columns of our FINAL DATAFRAME

In [13]:
df_final.columns

Index(['MMSID', 'editionTitle', 'editor', 'editor_date', 'genre', 'language',
       'termsOfAddress', 'numberOfPages', 'physicalDescription', 'place',
       'publisher', 'referencedBy', 'shelfLocator', 'editionSubTitle',
       'volumeTitle', 'year', 'volumeId', 'metsXML', 'permanentURL',
       'publisherPersons', 'volumeNum', 'letters', 'part', 'editionNum',
       'supplementTitle', 'supplementSubTitle', 'supplementsTo',
       'numberOfVolumes', 'term', 'definition', 'relatedTerms', 'header',
       'startsAt', 'endsAt', 'numberOfTerms', 'numberOfWords', 'positionPage',
       'typeTerm', 'altoXML'],
      dtype='object')

In [14]:
df_final

,MMSID,editionTitle,editor,editor_date,genre,language,termsOfAddress,numberOfPages,physicalDescription,place,...,definition,relatedTerms,header,startsAt,endsAt,numberOfTerms,numberOfWords,positionPage,typeTerm,altoXML
0,997902523804341,"Second edition, Volume 1, A-AST",None,None,encyclopedia,eng,None,872,"10v.,plates : maps ; 4to",Edinburgh,...,,[],&C,15,15,7,0,0,Article,144850370/alto/188386616.34.xml
1,997902523804341,"Second edition, Volume 1, A-AST",None,None,encyclopedia,eng,None,872,"10v.,plates : maps ; 4to",Edinburgh,...,&C. A A >. A THE charafter of the firftietter ...,[],&C,15,15,7,579,0,Article,144850370/alto/188386616.34.xml
2,997902523804341,"Second edition, Volume 1, A-AST",None,None,encyclopedia,eng,None,872,"10v.,plates : maps ; 4to",Edinburgh,...,"a little town in Germany, in the circle of Sua...",[],&C,15,15,7,54,0,Article,144850370/alto/188386616.34.xml
3,997902523804341,"Second edition, Volume 1, A-AST",None,None,encyclopedia,eng,None,872,"10v.,plates : maps ; 4to",Edinburgh,...,"a little town in Germany, in the circle of Wef...",[],&C,15,15,7,37,0,Article,144850370/alto/188386616.34.xml
4,997902523804341,"Second edition, Volume 1, A-AST",None,None,encyclopedia,eng,None,872,"10v.,plates : maps ; 4to",Edinburgh,...,"a Dutch measure of capacity for liquids, conta...",[],&C,15,15,7,13,0,Article,144850370/alto/188386616.34.xml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16660,997902523804341,"Second edition, Volume 10, SCU-Appendix",None,None,encyclopedia,eng,None,1280,"10v.,plates : maps ; 4to",Edinburgh,...,Bark. See the two following ar-,[],WIN,1262,1262,2,6,1,Article,190273290/alto/190333710.34.xml
16661,997902523804341,"Second edition, Volume 10, SCU-Appendix",None,None,encyclopedia,eng,None,1280,"10v.,plates : maps ; 4to",Edinburgh,...,[Encyc L) The art of moulding wood is mentione...,[],WINWOO,1264,1264,1,651,1,Article,190273290/alto/190333736.34.xml
16662,997902523804341,"Second edition, Volume 10, SCU-Appendix",None,None,encyclopedia,eng,None,1280,"10v.,plates : maps ; 4to",Edinburgh,...,TERANTA Aromatic a.these harder parts of trees...,[],*,1267,1269,1,769,0,Article,190273290/alto/190333775.34.xml
16663,997902523804341,"Second edition, Volume 10, SCU-Appendix",None,None,encyclopedia,eng,None,1280,"10v.,plates : maps ; 4to",Edinburgh,...,"Stockdale’s manners, &c. of ancient nations, 2...",[],LISTAUTH,1272,1272,1,706,0,Topic,190273290/alto/190333840.34.xml


### 3. Saving new final dataframes to disk

In [15]:
df_final.to_json(r'../../results_NLS/final_eb_2_dataframe', orient="index")

### 4. Loading new final dataframes to memory

In [16]:
df = pd.read_json('../../results_NLS/final_eb_2_dataframe', orient="index") 

In [17]:
df.head(2)

,MMSID,editionTitle,editor,editor_date,genre,language,termsOfAddress,numberOfPages,physicalDescription,place,...,definition,relatedTerms,header,startsAt,endsAt,numberOfTerms,numberOfWords,positionPage,typeTerm,altoXML
0,997902523804341,"Second edition, Volume 1, A-AST",NaN,NaN,encyclopedia,eng,NaN,872,"10v.,plates : maps ; 4to",Edinburgh,...,,[],&C,15,15,7,0,0,Article,144850370/alto/188386616.34.xml
1,997902523804341,"Second edition, Volume 1, A-AST",NaN,NaN,encyclopedia,eng,NaN,872,"10v.,plates : maps ; 4to",Edinburgh,...,&C. A A >. A THE charafter of the firftietter ...,[],&C,15,15,7,579,0,Article,144850370/alto/188386616.34.xml
